In [1]:
pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 12.2MB/s 


In [2]:
from google.colab import drive

drive.mount('/content/drive')
%cd /content/drive/My\ Drive/ML\ Project/

Mounted at /content/drive
/content/drive/My Drive/ML Project


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import h5py
from PIL import Image
from matplotlib import pyplot as plt
from keras.preprocessing.image import array_to_img
import sys
import tensorflow_model_optimization as tfmot
import tempfile

## Load B1 Model with Parameters

In [4]:
B1_model = './github/models/sunglasses_bd_net.h5'
model_B1 = keras.models.load_model(B1_model)
B1_para = './github/models/sunglasses_bd_weights.h5'
model_B1.load_weights(B1_para)

model_B1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
____________________________________________________________________________________________

## Setting Pruning Rules

In [5]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

## Load Dataset

In [6]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))
    return x_data / 255, y_data

clean_validation_data_path = './github/data/clean_validation_data.h5'
clean_test_data_path = './github/data/clean_test_data.h5'
sunglasses_poisoned_data_path = './github/data/sunglasses_poisoned_data.h5'

x_data_valid, y_data_valid = data_loader(clean_validation_data_path)
x_data_test, y_data_test = data_loader(clean_test_data_path)
x_data_backdoor, y_data_backdoor = data_loader(sunglasses_poisoned_data_path)

## Pruning Operation

In [7]:
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.5,
                                    final_sparsity=0.8,
                                    begin_step=0,
                                    end_step=2000)
}

model_B1_after_pruning = prune_low_magnitude(model_B1, **pruning_params)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [8]:
model_B1_after_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_B1_after_pruning.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
____________________________________________________________________________________________

## Training Again

In [9]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
  keras.callbacks.EarlyStopping(monitor="val_loss", patience=0)
]

model_B1_after_pruning.fit(x_data_valid, y_data_valid, epochs=3, callbacks=callbacks)

Epoch 1/3
361/361 [==============================] - 36s 89ms/step - loss: 0.8816 - accuracy: 0.8287
Epoch 2/3
361/361 [==============================] - 32s 90ms/step - loss: 0.3762 - accuracy: 0.9054
Epoch 3/3
361/361 [==============================] - 32s 89ms/step - loss: 0.4044 - accuracy: 0.9005


## Load B1 Model with Parameters

In [15]:
B1_model = './github/models/sunglasses_bd_net.h5'
model_B1 = keras.models.load_model(B1_model)
B1_para = './github/models/sunglasses_bd_weights.h5'
model_B1.load_weights(B1_para)

model_B1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
____________________________________________________________________________________________

## Evaluating Function

In [10]:
def evaluate(model, x, y):
  pred = np.argmax(model.predict(x), axis=1)
  acc = np.mean(np.equal(pred, y))*100
  return acc

## Evaluation of Pruned B1 Model

In [18]:
print('model_B1_after_pruning valid accuracy:', evaluate(model_B1_after_pruning, x_data_valid, y_data_valid))

model_B1_after_pruning valid accuracy: 93.92915908894085


In [19]:
print('model_B1_after_pruning test accuracy:', evaluate(model_B1_after_pruning, x_data_test, y_data_test))

model_B1_after_pruning test accuracy: 81.12236944660951


## Evaluation of Original B1 Model

In [17]:
print('model_B1 valid accuracy:', evaluate(model_B1, x_data_valid, y_data_valid))

model_B1 valid accuracy: 97.88689702953148


In [16]:
print('model_B1 test accuracy:', evaluate(model_B1, x_data_test, y_data_test))

model_B1 test accuracy: 97.77864380358535


## Backdoor Data Test

In [20]:
p1 = np.argmax(model_B1.predict(x_data_backdoor), axis=1)
p2 = np.argmax(model_B1_after_pruning.predict(x_data_backdoor), axis=1)
print(p1)
print(p2)
n = 0
for a, b in zip(p1, p2):
  if a != b:
    n += 1
print('The number of cases we identify as backdoor data in the sunglasses dataset is:', n)

[0 0 0 ... 0 0 0]
[ 934 1079  583 ...  740 1064  915]
The number of cases we identify as backdoor data in the sunglasses dataset is: 12474


## Final Repaired B1 Model

In [21]:
def repaired_B1(x):
  y1 = np.argmax(model_B1.predict(x), axis=1)
  y2 = np.argmax(model_B1_after_pruning.predict(x), axis=1)
  res = []
  for i in range(len(x)):
    if y1[i] == y2[i]:
      res.append(y1[i])
    else:
      res.append(1283)
  return np.array(res)

## Evaluation of Repaired B1 Model

In [22]:
repaire_B1_valid_res = repaired_B1(x_data_valid)
repaire_B1_valid_acc = np.mean(np.equal(repaire_B1_valid_res, y_data_valid))*100
print('repaired B1 valid accuracy: ',repaire_B1_valid_acc)

repaired B1 valid accuracy:  92.36165237724084


In [23]:
repaire_B1_test_res = repaired_B1(x_data_test)
repaire_B1_test_acc = np.mean(np.equal(repaire_B1_test_res, y_data_test))*100
print('repaired B1 test accuracy: ',repaire_B1_test_acc)

repaired B1 test accuracy:  80.67809820732657


## Save Pruned B1 Model

In [25]:
model_B1_after_pruning.save('./Pruned_B1.h5')